In [57]:
from sklearn.datasets import load_boston
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
np.set_printoptions(suppress=True)
%matplotlib inline

In [146]:
def prepare_boston_data():
    # подгружаем boston dataset
    data = load_boston()
    X, y = data['data'], data['target']

    # нормализуем X
    X = (X - np.mean(X, axis=0))/np.std(X, axis=0)

    # добавляем столбец свободных членов (bias линейной модели)
    n = X.shape[0]  # количество строк
    X = np.hstack([np.ones(n).reshape(n, 1), X])

    return X, y

def print_regression_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    print(f'mse: {mse:.2f}, rmse: {rmse:.2f}')

Обернем написанную нами линейную регрессию методом матричных операций в класс:

In [59]:
class LinRegAlgebra():
    def __init__(self):
        self.theta = None
        self.X = None
            
    def fit(self, X, y):
        self.theta = (np.linalg.inv((X.T).dot(X))).dot((X.T).dot(y))
            
    def predict(self, X):
        if self.theta is None:
            raise Exception("You should train the model first")
        return X.dot(self.theta)

In [191]:
# пробуем модель на всем датасете
X, y = prepare_boston_data()
model = LinRegAlgebra()
model.fit(X, y)
y_pred = model.predict(X)
print_regression_metrics(y,y_pred)

mse: 21.89, rmse: 4.68


In [61]:
# пробуем модель на train_test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
model_train = LinRegAlgebra()
model_train.fit(X_train,y_train)
y_pred = model.predict(X_test)
print_regression_metrics(y_test,y_pred)

mse: 23.50, rmse: 4.85


Сделаем базовый абстрактный класс для регрессии:

In [136]:
class RegOptimizer():
    def __init__(self, alpha, n_iters):
        self._theta = None
        self._alpha = alpha
        self._n_iters = n_iters
    
    def fit(self, X, y, start_theta, message=True):
        self._theta = self.optimize(X, y, start_theta, message)
    
    def optimize(self, X, y, start_theta):
        raise NotImplementedError()

    def grad_func(self, X, y, theta):
        raise NotImplementedError()
    
    def gradient_step(self, theta, alpha, theta_grad):
        return theta - alpha*theta_grad

    def predict(self, X):
        if self._theta is None:
            raise Exception('You should train the model first')
        return X.dot(self._theta)

А теперь класс с моделью классической линейной регрессии

In [137]:
class LinReg(RegOptimizer):
    
    def optimize(self, X, y, start_theta, message): # функция градиентного спуска по кол-ву итераций
        theta = start_theta.copy()
                
        for i in range(self._n_iters):
            theta_grad = self.grad_func(X, y, theta)
            theta = self.gradient_step(theta, theta_grad, self._alpha)
            
        return theta
        
    def grad_func(self, X, y, theta): # конкретизируем градиент для классической линейной регрессии
        n = X.shape[0]
        return (1/n)*(X.T).dot(X.dot(theta)-y)

In [138]:
alpha = 0.01
n_iters = 1000
start_theta = np.ones(X.shape[1])

model_grad = LinReg(alpha, n_iters)
model_grad.fit(X, y, start_theta)
y_pred = model_grad.predict(X)

print_regression_metrics(y, y_pred)

mse: 22.21, rmse: 4.71


Сделайте для градиентного спуска остановку алгоритма, если максимальное из абсолютных значений компонент градиента становится меньше 0.01. Для градиентного спуска установите alpha = 0.2.

На какой итерации останавливается градиентный спуск?

In [139]:
class LinRegFast(RegOptimizer):
    
    def optimize(self, X, y, start_theta, message): # функция градиентного спуска по кол-ву итераций
        theta = start_theta.copy()
                
        for i in range(self._n_iters):
            theta_grad = self.grad_func(X, y, theta)
            theta = self.gradient_step(theta, theta_grad, self._alpha)
            if np.max(theta_grad) < 0.01:
                break
        
        if message:
            print('Model training completed successfully')
            print(f'Number of iterations: {i}')
            
        return theta
        
    def grad_func(self, X, y, theta): # конкретизируем градиент для классической линейной регрессии
        n = X.shape[0]
        return (1/n)*(X.T).dot(X.dot(theta)-y)

In [142]:
alpha = 0.2
n_iters = 1000
start_theta = np.ones(X.shape[1])

model_grad_fast = LinRegFast(alpha, n_iters)
model_grad_fast.fit(X, y, start_theta)
y_pred = model_grad_fast.predict(X)

print_regression_metrics(y, y_pred)

Model training completed successfully
Number of iterations: 212
mse: 21.90, rmse: 4.68


In [145]:
# Сравните скорость обучения градиентным спуском и матричными операциями.
model = LinRegAlgebra()
print('Матричный метод: ')
%timeit model.fit(X, y)

model_grad = LinReg(alpha, n_iters)
print('Градиентный спуск 1000 итераций: ')
%timeit model_grad.fit(X, y, start_theta)

model_grad_fast = LinRegFast(alpha, n_iters)
print('Градиентный спуск по условию: ')
%timeit model_grad_fast.fit(X, y, start_theta, message=False)


Матричный метод: 
73.2 µs ± 561 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
Градиентный спуск 1000 итераций: 
9.15 ms ± 32.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Градиентный спуск по условию: 
3.2 ms ± 22.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Добавьте к признакам нелинейной модели квадрат признака DIS и переобучите модель. Какой получился RMSE? Подсказка: используйте написанную нами линейную регрессию методом матричных операций.

In [227]:
def prepare_boston_data_new():
    # подгружаем boston dataset
    data = load_boston()
    X, y = data['data'], data['target']
    
    # добавляем к признакам нелинейной модели квадрат признака DIS
    ind = np.where(data.feature_names=='DIS')[0] # индекс столбца с признаком DIS
    #X = np.hstack([X,(X[:,ind])**2])
    
    X = np.hstack([X, np.sqrt(X[:, 5:6]), X[:, 6:7] ** 3, (X[:,ind])**2])
    
    # нормализуем X
    X = (X - np.mean(X, axis=0))/np.std(X, axis=0)
    
    # добавляем столбец свободных членов (bias линейной модели)
    n = X.shape[0]  # количество строк
    X = np.hstack([np.ones(n).reshape(n, 1), X])
    
    return X, y

In [205]:
X, y, data = prepare_boston_data_new()
display(pd.DataFrame(X))
display(data['feature_names'])
pd.DataFrame(data['data'])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1.0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,16.728100
1,1.0,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,24.672082
2,1.0,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,24.672082
3,1.0,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,36.750269
4,1.0,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.750269
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,1.0,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67,6.143458
502,1.0,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08,5.232656
503,1.0,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64,4.698056
504,1.0,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48,5.706843


array(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD',
       'TAX', 'PTRATIO', 'B', 'LSTAT'], dtype='<U7')

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48


In [223]:
X, y = prepare_boston_data_new()

model = LinRegAlgebra()
model.fit(X,y)
y_pred = model.predict(X)
print_regression_metrics(y, y_pred)

mse: 16.06, rmse: 4.01


In [212]:
def train_validate(X, y):
    # Разбить данные на train/valid
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=1)

    # Создать и обучить линейную регрессию
    linreg_alg = LinRegAlgebra()
    linreg_alg.fit(X_train, y_train)

    # Сделать предсказания по валидционной выборке
    y_pred = linreg_alg.predict(X_valid)

    # Посчитать значение ошибок MSE и RMSE для валидационных данных
    print_regression_metrics(y_valid, y_pred)

In [226]:
# Подготовить данные без модификации признаков
X, y = prepare_boston_data_new()
# Провести эксперимент
train_validate(X, y)

mse: 13.59, rmse: 3.69


In [228]:
def prepare_boston_data_new2():
    # подгружаем boston dataset
    data = load_boston()
    X, y = data['data'], data['target']
    
    X = np.hstack([X, np.sqrt(X[:, 5:6]), X[:, 6:7] ** 3])
    
    # нормализуем X
    #X = (X - np.mean(X, axis=0))/np.std(X, axis=0)
    
    # добавляем столбец свободных членов (bias линейной модели)
    n = X.shape[0]  # количество строк
    X = np.hstack([np.ones(n).reshape(n, 1), X])
    
    return X, y

In [229]:
# Подготовить данные без модификации признаков
X, y = prepare_boston_data_new2()
# Провести эксперимент
train_validate(X, y)

mse: 14.28, rmse: 3.78
